# TensorFlow csv文件读取数据（代码实现）详解
大多数人了解 Pandas 及其在处理[大数据](http://c.biancheng.net/big_data/)文件方面的实用性。TensorFlow 提供了读取这种文件的方法。

前面章节中，介绍了如何在 TensorFlow 中读取文件，本节将重点介绍如何从 CSV 文件中读取数据并在训练之前对数据进行预处理。

本节将采用哈里森和鲁宾菲尔德于 1978 年收集的波士顿房价数据集（http://lib.stat.cmu.edu/datasets/boston ），该数据集包括 506 个样本场景，每个房屋含 14 个特征：
1. CRIM：城镇人均犯罪率
2. ZN：占地 25000 平方英尺（1 英尺=0.3048 米）以上的住宅用地比例
3. INDUS：每个城镇的非零售商业用地比例
4. CHAS：查尔斯河（Charles River）变量（若土地位于河流边界，则为 1；否则为 0）
5. NOX：一氧化氮浓度（每千万）
6. RM：每个寓所的平均房间数量
7. AGE：1940 年以前建成的自住单元比例
8. DIS：到 5 个波士顿就业中心的加权距离
9. RAD：径向高速公路可达性指数
10. TAX：每万美元的全价值物业税税率
11. PTRATIO：镇小学老师比例
12. B：1000(Bk-0.63)2，其中 Bk 是城镇黑人的比例
13. LSTAT：低地位人口的百分比
14. MEDV：1000 美元自有住房的中位值

## TensorFlow读取csv文件过程
1. 导入所需的模块并声明全局变量：

In [1]:
import tensorflow as tf

# 全局变量
DATA_FILE = '../Database/boston_housing.csv'
BATCH_SIZE = 10 # 批次的大小
NUM_FEATURES = 14 # 特征的数量

2. 定义一个将文件名作为参数的函数，并返回大小等于 BATCH_SIZE 的张量：

In [2]:
def data_generator(filename):
    """ 生成器张量批量Batch_SIZE批量。
    Args: 字符串张量 要从中读取数据的文件名
    Returns: feature_batch和label_batch
    """
    # 3.定义 f_queue(文件队列) 和 reader 为文件名：
    f_queue = tf.train.string_input_producer(filename) # 使用tf.train.string_input_producer创建文件名队列
    # reader对象从文件名队列中读取数据
    reader = tf.TextLineReader(skip_header_lines=1) # skips the first line 因为第一行为标签
    _, value = reader.read(f_queue)
    print("_:",_,"\nvalue:",value)
    
    # 4.这里指定要使用的数据以防数据丢失。对 .csv 解码并选择需要的特征。例如，选择 RM、PTRATIO 和 LSTAT 特征：
    record_defaults = [[0.0] for _ in range(NUM_FEATURES)]
    data = tf.decode_csv(value,record_defaults=record_defaults)
    print("data:",data,"\n")
    features = tf.stack(tf.gather_nd(data,[[5],[10],[12]]))
    label = data[-1]
    
    # 5.定义参数来生成批并使用 tf.train.shuffle_batch() 来随机重新排列张量。该函数返回张量 feature_batch 和 label_batch：
    # minimum number elements in the queue after a dequeue
    min_after_dequeue = 10 * BATCH_SIZE
    # the maximum number of elements in the queue (队列中的最大元素数)
    capacity = 20 * BATCH_SIZE
    
    
    # shuffile the data to generate BATCH_SIZE sample pairs(将数据移动以生成BATCH_SIZE样本对)
    feature_batch,label_batch = tf.train.shuffle_batch([features,label],
                                                       batch_size=BATCH_SIZE,
                                                       capacity=capacity,
                                                       min_after_dequeue=min_after_dequeue)
    return feature_batch,label_batch

6. 这里定义了另一个函数在会话中生成批：

In [3]:
def generate_data(feature_batch,label_batch):
    with tf.Session() as sess:
        # intialize the queue threads (初始化队列线程) 
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord) # 启动填充为文件队列的线程
        for _ in range(5): # Generate 5 batches
            features, labels = sess.run([feature_batch,label_batch])
            print(features, 'HI')
        coord.request_stop()
        coord.join(threads)

7. 使用这两个函数得到批中的数据。这里，仅打印数据；在学习训练时，将在这里执行优化步骤：

In [4]:
feature_batch, label_batch = data_generator([DATA_FILE])
generate_data(feature_batch,label_batch)

W0808 18:18:32.234528 32824 deprecation.py:323] From <ipython-input-2-eaf54aa1157c>:7: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0808 18:18:32.278502 32824 deprecation.py:323] From D:\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\training\input.py:278: input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0808 18:18:32.281

_: Tensor("ReaderReadV2:0", shape=(), dtype=string) 
value: Tensor("ReaderReadV2:1", shape=(), dtype=string)
data: [<tf.Tensor 'DecodeCSV:0' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:1' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:2' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:3' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:4' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:5' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:6' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:7' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:8' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:9' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:10' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:11' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:12' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:13' shape=() dtype=float32>] 

[[ 5.594 18.9   13.09 ]
 [ 7.079 17.8    5.7  ]
 [ 6.096 21.    20.34 ]
 [ 6.575 15.3    4.98 ]
 [ 5.787 16.1   10.24 ]
 [ 6.249 18.2   10.59 ]
 [ 6.121 18.5    8.44 ]
 [ 5.851 2

# TensorFlow csv数据预处理
用前面章节提到的 TensorFlow 控制操作和张量来对数据进行预处理。例如，对于波士顿房价的情况，大约有 16 个数据行的 MEDV 是 50.0。在大多数情况下，这些数据点包含缺失或删减的值，因此建议不要考虑用这些数据训练。可以使用下面的代码在训练数据集中删除它们：

In [5]:
condition = tf.equal(data[13],tf.constant(50.0))
data = tf.where(condition, tf.zeros(NUM_FEATURES),data[:])

NameError: name 'data' is not defined

这里定义了一个张量布尔条件，若 MEDV 等于 50.0 则为真。如果条件为真则可使用 TensorFlow tf.where() 操作赋为零值。